In [1]:
import json
import logging
import multiprocessing
from pathlib import Path
from typing import Any, Dict, List, Tuple

from dask.callbacks import Callback
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg
import geopandas as gpd
import numpy as np
from omegaconf import DictConfig, OmegaConf
import pandas as pd
from scipy.sparse import csr_matrix
from shapely.geometry import LineString, MultiLineString, Point
from tqdm import tqdm
import utm
import xarray as xr
import zarr

import sys
sys.path.append("..")

from marquette.merit._graph import _find_flowlines

log = logging.getLogger(__name__)

from dask.distributed import Client

client = Client(dashboard_address=':8989')
client
### TODO:
# - Preprocessing functions for catchment area of MERIT basins
# - Preprocessing function for flowline geometry (using custom UTM zones)
# - Preprocess function for the number of edges (the DX/buffer)
# - Create Edges

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8989/status,
Dashboard: http://127.0.0.1:8989/status,Workers: 12
Total threads: 144,Total memory: 503.74 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34665,Workers: 12
Dashboard: http://127.0.0.1:8989/status,Total threads: 144
Started: Just now,Total memory: 503.74 GiB
Comm: tcp://127.0.0.1:39971,Total threads: 12
Dashboard: http://127.0.0.1:45251/status,Memory: 41.98 GiB
Nanny: tcp://127.0.0.1:40845,


In [4]:
json_data = '''{
  "name": "MERIT",
  "data_path": "/data/tkb5476/projects/marquette/data/",
  "dx": 2000,
  "buffer": 0.3334,
  "continent": 7,
  "area": 8,
  "save_name": "${name}_${continent}${area}",
  "save_paths": {
    "flow_lines": "${data_path}/${name}/raw/flowlines"
  },
  "zarr": {
    "edges": "${data_path}/${name}/zarr/dpl_v2/${save_name}_edges/",
    "sorted_edges_keys": "${data_path}/${name}/zarr/dpl_v2/${save_name}_edge_keys/"
  }
}'''

data_dict = json.loads(json_data)
cfg = OmegaConf.create(data_dict)


In [5]:
def _plot_gdf(gdf: gpd.GeoDataFrame) -> None:
    """
    A function to find the correct flowline of all MERIT basins using glob

    Parameters
    ----------
    gdf : gpd.GeoDataFrame
        The geodataframe you want to plot

    Returns
    -------
    None

    Raises
    ------
    None
    """
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax)
    ax.set_title("Polyline Plot")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.show()

In [6]:
def _find_flowlines(cfg: DictConfig) -> Path:
    """
    A function to find the correct flowline of all MERIT basins using glob

    Parameters
    ----------
    cfg : DictConfig
        The cfg object

    Returns
    -------
    Path
        The file that we're going to create flowline connectivity for

    Raises
    ------
    IndexError
        Raised if no flowlines are found with your MERIT region code
    """
    flowline_path = Path(cfg.save_paths.flow_lines)
    region_id = f"_{cfg.continent}{cfg.area}_"
    matching_file = flowline_path.glob(f"*{region_id}*.shp")
    try:
        found_file = [file for file in matching_file][0]
        return found_file
    except IndexError as e:
        raise IndexError(f"No flowlines found using: *{region_id}*.shp")

# Segments

In [8]:
def create_segment_dict(row: pd.Series, segment_coords: List[Tuple[float, float]], crs: Any, dx: int, buffer: float) -> Dict[str, Any]:
    """
    Create a dictionary representation of a segment with various attributes.

    This function constructs a dictionary for a river segment based on provided
    attributes. It includes details such as segment ID, order, length, downstream
    ID, slope, sinuosity, stream drop, upstream area, coordinates, and CRS.

    Parameters
    ----------
    row : pandas.Series
        A series representing a row from a DataFrame containing segment data.
    segment_coords : List[Tuple[float, float]]
        List of tuples representing coordinates of the segment.
    crs : Any
        Coordinate reference system of the segment.
    dx : int
        Desired length of each edge in the segment (used in further calculations).
    buffer : float
        Buffer tolerance for edge length calculation.

    Returns
    -------
    Dict[str, Any]
        Dictionary containing segment attributes.
    """
    segment_dict = {
        'id': row["COMID"],
        'order': row["order"],
        'len': row["lengthkm"] * 1000,  # to meters
        'len_dir': row["lengthdir"] * 1000,  # to meters
        'ds': row["NextDownID"],
        # 'is_headwater': False,
        'up': [row[key] for key in ["up1", "up2", "up3", "up4"] if row[key] != 0] if row["maxup"] > 0 else ([] if row["order"] == 1 else []),
        'slope': row["slope"],
        'sinuosity': row["sinuosity"],
        'stream_drop': row["strmDrop_t"],
        'uparea': row["uparea"],
        'coords': segment_coords,
        'crs': crs,
    }

    return segment_dict

In [9]:
def create_segment(row: pd.Series, crs: Any, dx: int, buffer: float) -> Dict[str, Any]:
    """
    Create a dictionary representation of a segment using its row data.

    This function is a wrapper that calls 'create_segment_dict' by passing the
    geometry of the segment along with other attributes. It simplifies the creation
    of a segment dictionary from a DataFrame row.

    Parameters
    ----------
    row : pandas.Series
        A series representing a row from a DataFrame containing segment data.
    crs : Any
        Coordinate reference system of the segment.
    dx : int
        Desired length of each edge in the segment (used in further calculations).
    buffer : float
        Buffer tolerance for edge length calculation.

    Returns
    -------
    dict
        Dictionary containing segment attributes.
    """
    return create_segment_dict(row, row.geometry, crs, dx, buffer)

In [10]:
def calculate_num_edges(length: float, dx: float, buffer: float) -> Tuple:
    """
    Calculate the number of edges and the length of each edge for a given segment.

    This function determines the number of edges a segment should be divided into, 
    based on its length, a desired edge length (dx), and a tolerance (buffer). 
    The function adjusts the number of edges to ensure that the deviation of the 
    actual edge length from dx is within the specified buffer.

    Parameters
    ----------
    length : float
        The length of the segment for which to calculate the number of edges.
    dx : float
        The desired length of each edge.
    buffer : float
        The acceptable deviation from the desired edge length (dx).

    Returns
    -------
    tuple
        A tuple containing two elements:
            - The first element is an integer representing the number of edges.
            - The second element is a float representing the actual length of each edge.

    Examples
    --------
    >> calculate_num_edges(100, 30, 5)
    (3, 33.333333333333336)

    >> calculate_num_edges(100, 25, 2)
    (4, 25.0)
    """
    num_edges = length // dx
    if num_edges == 0:
        num_edges = 1
        if dx - length < buffer:
            edge_len = length
        else:
            edge_len = dx
    else:
        edge_len = length / num_edges
        buf_dev = edge_len - dx
        while abs(buf_dev) > buffer:
            if buf_dev > dx:
                num_edges -= 1
            else:
                num_edges += 1
            edge_len = length / num_edges
            buf_dev = edge_len - dx
    return (int(num_edges), edge_len)

# Edges

In [11]:
def create_edge_json(segment_row: pd.Series, up=None, ds=None, edge_id=None) -> Dict[str, Any]:
    """
    Create a JSON representation of an edge based on segment data.

    Parameters
    ----------
    segment_row : pandas.Series
        A series representing a row from the segment DataFrame.
    up : list, optional
        List of upstream segment IDs.
    ds : str, optional
        Downstream segment ID.
    edge_id : str, optional
        Unique identifier for the edge.

    Returns
    -------
    dict
        Dictionary representing the edge with various attributes.
    """
    edge = {
        'id': edge_id,
        'merit_basin': segment_row['id'],
        'segment_sorting_index': segment_row['index'],
        'order': segment_row['order'],
        'len': segment_row['len'],
        'len_dir': segment_row['len_dir'],
        'ds': ds,
        'up': up,
        'slope': segment_row['slope'],
        'sinuosity': segment_row['sinuosity'],
        'stream_drop': segment_row['stream_drop'],
        'uparea': segment_row['uparea'],
        'coords': segment_row['coords'],
        'crs': segment_row['crs'],
    }
    return edge

def calculate_drainage_area(edge: Dict[str, Any], idx: int, segment_das: Dict[str, float]) -> None:
    """
    Calculate the drainage area for an edge.

    Parameters
    ----------
    edge : dict
        Dictionary representing the edge.
    idx : int
        Index of the edge within the segment.
    segment_das : dict
        Dictionary containing drainage area data for each segment.

    Returns
    -------
    None
        The function modifies the 'edge' dictionary in place, adding or updating
        the 'uparea' key with the calculated drainage area.
    """
    prev_up_area = 0
    if edge['up']:
        try:
            prev_up_area = sum(segment_das[seg] for seg in edge['up'])
        except KeyError:
            prev_up_area = 0
            log.info("Missing upstream branch. Treating as head node")
        ratio = (edge["len"] * (idx + 1)) / edge["len_dir"]
        area_difference = edge['uparea'] - prev_up_area
        edge["uparea"] = prev_up_area + (area_difference * ratio)

def get_upstream_ids(row: pd.Series, edge_counts: int):
    """
    Generate upstream IDs for a segment.

    Parameters
    ----------
    row : pandas.Series
        A series representing a row from the segment DataFrame.
    edge_counts : int
        The number of edges associated with the segment.

    Returns
    -------
    list
        List of upstream segment IDs.
    """
    if row['up'] is None:
        return []
    try:
        up_ids = [f"{up}_{edge_counts - 1}" for up in row['up']]
    except KeyError:
        log.error(f"KeyError with segment {row['id']}")
        return []
    return up_ids

In [12]:
def singular_segment_to_edge_partition(df: pd.DataFrame, edge_info: Dict[str, Any], segment_das: Dict[str, float]) -> pd.DataFrame:
    """
    Process a DataFrame partition to create edges for each segment.

    This function iterates over each segment in the DataFrame, computes the edge 
    length, upstream IDs, and creates JSON representation of each edge. It handles 
    segments that are associated with only one edge.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame partition containing segment data.
    edge_info : dict
        Dictionary containing edge information for each segment.
    segment_das : dict
        Dictionary containing drainage area data for each segment.

    Returns
    -------
    pandas.DataFrame
        DataFrame containing edge data for all segments in the partition.
    """
    all_edges = []
    num_edges = 1
    for _, segment in tqdm(df.iterrows(), total=len(df)):
        edge_len = edge_info[segment['id']][1]
        up_ids = get_upstream_ids(segment, num_edges)
        edge = create_edge_json(
            segment,
            up=up_ids,
            ds=f"{segment['ds']}_0",
            edge_id=f"{segment['id']}_0",
        )
        edge["len"] = edge_len
        edge["len_dir"] = edge_len / segment["sinuosity"]
        all_edges.append(edge)
    return pd.DataFrame(all_edges)

def many_segment_to_edge_partition(df: pd.DataFrame, edge_info: Dict[str, Any], segment_das: Dict[str, float]) -> pd.DataFrame:
    """
    Process a DataFrame partition to create edges for segments with multiple edges.

    This function iterates over each segment in the DataFrame partition, computes
    the edge length, upstream IDs, and creates a JSON representation for each edge.
    It is specifically designed for segments that have multiple edges.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame partition containing segment data.
    edge_info : dict
        Dictionary containing information about the number of edges and edge length 
        for each segment.
    segment_das : dict
        Dictionary containing drainage area data for each segment.

    Returns
    -------
    pandas.DataFrame
        DataFrame containing edge data for all segments in the partition.
    """
    all_edges = []
    for _, segment in tqdm(df.iterrows(), total=len(df), desc="Processing Segments"):
        num_edges, edge_len = edge_info[segment['id']]
        up_ids = get_upstream_ids(segment, num_edges)
        for i in range(num_edges):
            if i == 0:
                edge = create_edge_json(
                    segment,
                    up=up_ids,
                    ds=f"{segment['id']}_{i + 1}",
                    edge_id=f"{segment['id']}_{i}",
                )
            else:
                edge = create_edge_json(
                    segment,
                    up=[f"{segment['id']}_{i - 1}"],
                    ds=f"{segment['id']}_{i + 1}" if i < num_edges - 1 else f"{segment['ds']}_0",
                    edge_id=f"{segment['id']}_{i}",
                )
            edge["len"] = edge_len
            edge["len_dir"] = edge_len / segment["sinuosity"]
            calculate_drainage_area(edge, i, segment_das)
            all_edges.append(edge)
    return pd.DataFrame(all_edges)

# The functions
### Read in the polylines and convert to dask dataframe

In [13]:
flowline_file: Path = _find_flowlines(cfg)
polyline_gdf: gpd.GeoDataFrame = gpd.read_file(flowline_file)
dx: int = cfg.dx  # Unit: Meters
buffer: float = cfg.buffer * dx  # Unit: Meters
for col in [
    "COMID",
    "NextDownID",
    "up1",
    "up2",
    "up3",
    "up4",
    "maxup",
    "order",
]:
    polyline_gdf[col] = polyline_gdf[col].astype(int)
crs: Any = polyline_gdf.crs
dask_gdf: dg.GeoDataFrame = dg.from_geopandas(polyline_gdf, npartitions=48) 

### Create segments and find the ordering of the segments by drainage area

In [14]:
meta = pd.Series([], dtype=object)
with ProgressBar():
    computed_series: dd.Series = dask_gdf.map_partitions(
        lambda df: df.apply(create_segment, args=(polyline_gdf.crs, dx, buffer), axis=1),
        meta=meta
    ).compute()
    
segments_dict = computed_series.to_dict()
sorted_keys = sorted(segments_dict, key=lambda key: segments_dict[key]['uparea'])
segment_das = {segment['id']: segment['uparea'] for segment in segments_dict.values()}

/home/tkb5476/mambaforge/envs/marquette/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 87.69 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [18]:
num_edges_dict = {segment_["id"]: calculate_num_edges(segment_["len"], dx, buffer) for seg_id, segment_ in tqdm(segments_dict.items(), desc="Processing Number of Edges")}
one_edge_segment = {seg_id: edge_info for seg_id, edge_info in tqdm(num_edges_dict.items(), desc="Filtering Segments == 1") if edge_info[0] == 1}
more_than_one_edge = {seg_id: edge_info for seg_id, edge_info in tqdm(num_edges_dict.items(), desc="Filtering Segments > 1") if edge_info[0] > 1} 

Filtering Segments > 1: 100%|████████████████████████████████████████████████████████████████████████████████████| 28489/28489 [00:00<00:00, 2823256.94it/s]


In [19]:
segments_with_more_than_one_edge = {}
segments_with_one_edge = {}

for i, segment in segments_dict.items():
    segment_id = segment["id"]
    segment["index"] = i
    
    if segment_id in more_than_one_edge:
        segments_with_more_than_one_edge[segment_id] = segment
    elif segment_id in one_edge_segment:
        segments_with_one_edge[segment_id] = segment
    else:
        print(f"MISSING ID: {segment_id}")

df_one = pd.DataFrame.from_dict(segments_with_one_edge, orient='index')
df_many = pd.DataFrame.from_dict(segments_with_more_than_one_edge, orient='index')
ddf_one = dd.from_pandas(df_one, npartitions=48)
ddf_many = dd.from_pandas(df_many, npartitions=48)

In [20]:
many_segment_to_edge_partition(df_many, many_edge_segment, segment_das).head()

Processing Segments: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 22690/22690 [00:05<00:00, 4393.27it/s]


,id,merit_basin,segment_sorting_index,order,len,len_dir,ds,up,slope,sinuosity,stream_drop,uparea,coords,crs
0,78000001_0,78000001,0,3,2560.284421,1742.964670,78000001_1,"[78000002_2, 78000003_2]",0.005052,1.468925,38.9,1842.457826,LINESTRING (-133.15583333333333 59.00666666666...,EPSG:4326
1,78000001_1,78000001,0,3,2560.284421,1742.964670,78000001_2,[78000001_0],0.005052,1.468925,38.9,3684.915652,LINESTRING (-133.15583333333333 59.00666666666...,EPSG:4326
2,78000001_2,78000001,0,3,2560.284421,1742.964670,78000369_0,[78000001_1],0.005052,1.468925,38.9,5527.373478,LINESTRING (-133.15583333333333 59.00666666666...,EPSG:4326
3,78000002_0,78000002,1,3,2043.085156,1511.372348,78000002_1,"[78000399_9, 78000627_9]",0.007258,1.351808,148.7,671.234704,"LINESTRING (-133.215 59.042500000000004, -133....",EPSG:4326
4,78000002_1,78000002,1,3,2043.085156,1511.372348,78000002_2,[78000002_0],0.007258,1.351808,148.7,1342.469408,"LINESTRING (-133.215 59.042500000000004, -133....",EPSG:4326


### Processing flowline segments to river graph edges

In [21]:
meta = pd.DataFrame({
    'id': pd.Series(dtype='str'),
    'merit_basin': pd.Series(dtype='int'),
    'segment_sorting_index': pd.Series(dtype='int'),
    'order': pd.Series(dtype='int'),
    'len': pd.Series(dtype='float'),
    'len_dir': pd.Series(dtype='float'),
    'ds': pd.Series(dtype='str'),
    'up': pd.Series(dtype='object'),  # List or array
    'slope': pd.Series(dtype='float'),
    'sinuosity': pd.Series(dtype='float'),
    'stream_drop': pd.Series(dtype='float'),
    'uparea': pd.Series(dtype='float'),
    'coords': gpd.GeoSeries(dtype='geometry'),  # Assuming this is a geometry column
    'crs': pd.Series(dtype='object'),  # CRS object
})

edges_results_one = ddf_one.map_partitions(
    singular_segment_to_edge_partition,
    edge_info=one_edge_segment, 
    segment_das=segment_das,
    meta=meta
)
edges_results_many = ddf_many.map_partitions(
    many_segment_to_edge_partition,
    edge_info=many_edge_segment, 
    segment_das=segment_das,
    meta=meta
)

In [22]:
edges_results_one_df = edges_results_one.compute()
edges_results_many_df = edges_results_many.compute()

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]


  0%|          | 0/121 [00:00<?, ?it/s]


100%|██████████| 120/120 [00:00<00:00, 3744.22it/s]


100%|██████████| 121/121 [00:00<00:00, 7581.24it/s]


  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]


  0%|          | 0/121 [00:00<?, ?it/s]

100%|██████████| 121/121 [00:00<00:00, 5367.70it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]


  0%|          | 0/121 [00:00<?, ?it/s]



  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]


  0%|          | 0/121 [00:00<?, ?it/s]



  0%|          | 0/121 [00:00<?, ?it/s]1559.46it/s]

100%|██████████| 121/121 [00:00<00:00, 1291.26it/s]



  0%|          | 0/120 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:00<00:00, 2073.13it/s]
/home/tkb5476/

In [23]:
merged_df = pd.concat([edges_results_one_df, edges_results_many_df])
for col in ["id", "ds", "up", "coords", "crs"]:
    merged_df[col] = merged_df[col].astype(str)
print(merged_df.dtypes)

id                        object
merit_basin                int64
segment_sorting_index      int64
order                      int64
len                      float64
len_dir                  float64
ds                        object
up                        object
slope                    float64
sinuosity                float64
stream_drop              float64
uparea                   float64
coords                    object
crs                       object
dtype: object


In [24]:
xr_dataset = xr.Dataset.from_dataframe(merged_df)
zarr_dataset = xr_dataset.to_zarr(Path(cfg.zarr.edges), mode='w')

In [25]:
sorted_keys_array = np.array(sorted_keys)
zarr.save(cfg.zarr.sorted_edges_keys, sorted_keys_array)